# Simulation of TLS

In [138]:
""" # Please remove the triple quotes in this line if the pip installation of the libraries is needed
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
%pip install pycryptodome
%pip install cryptography
%pip install requests --disable-pip-version-check """;

1. Client side

In [139]:
import random
from Crypto.Random import get_random_bytes

# generate random nonce, size 32 bytes
nonce_client = get_random_bytes(32)
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_client=random.randint(0,1)
print("Nonce-client: ",nonce_client)
print("Exchange method-client:",exchange_method_client)

Nonce-client:  b'fr\xf4\x19\x99\xb1\xa1\xf6\xfa1\xe0\xaa\xf1\xcd\x19\x00\x1ckZ\xec\rO\xd1\xb0\xb3\xd2\xec\xd1q\x82\x85\x11'
Exchange method-client: 0


2. Server side

In [140]:
# generate random nonce, size 32 bytes
nonce_server = get_random_bytes(32)
# specify exchange method → 0=Diffie-Hellman / 1=RSA
exchange_method_server=random.randint(0,1)
print("Nonce-server: ",nonce_server)
print("Exchange method-server:",exchange_method_server)

Nonce-server:  b"\x1dY\xf7\\\\=[%\xd8E\xd7\x81'^\xcdr\xb6\xaa\xc9\xad+$[\xe9\xb0\xfdIr\xd8\x16\xa02"
Exchange method-server: 1


Generate RSA KEY pair

In [141]:
from Crypto.PublicKey import RSA
# generate RSA key pair
key = RSA.generate(2048)
print(key.public_key().export_key())
# print(f"Public key:  (n={hex(key.n)}, e={hex(key.e)})")
# print(f"Private key: (n={hex(key.n)}, d={hex(key.d)})")

b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA3xIPjckWxunxicuAngIK\n2IDJN5vAxOTsdyG+ZsX0Bule4gZ33DpUaxcZDB3ATVNhm+Ok6Lt3JUVm+w8/WN4Z\n0Ld2IbG25ls7YReh8lNBIrdFeMUJF6gWbZZoB8jctECe9ZjBMNx5JAXvzAlEJVdg\nMF+jDC8FMMDwLP7mAfuIGwjoHJGbu5JsIzrb1RGsPadfInpmurKTdN88mp0/dH+8\nEYNhqQhutgF2id4Fbj8cgX+4LKK1fzF62LQAUUyuVbpJ7rDTDbn3mdsK8MtYznKn\nDZ3tJ4vPPkMAeuoDo9lHzuFdWA+lYzpHZM7d0/XUoZqrIs29mfu6algb1phYtj6c\niQIDAQAB\n-----END PUBLIC KEY-----'


Key hashing

In [142]:
import hashlib
from hashlib import sha256

# Hash key 
pk_client=key.public_key().export_key()
my_sha = hashlib.sha256(pk_client)
hash_client = int.from_bytes(my_sha.digest(),byteorder='big')
print("Sent to client: {public key, hash(public key)}. ",'\n'+'↓↓↓↓↓↓↓↓↓↓↓↓','\n')
print("Public key :",'\n',pk_client.decode(),'\n')
print("Generated hash :",'\n',hash_client)
# signature = pow(hash, key.d, key.n)
# print("Signature:", hex(signature))

Sent to client: {public key, hash(public key)}.  
↓↓↓↓↓↓↓↓↓↓↓↓ 

Public key : 
 -----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEA3xIPjckWxunxicuAngIK
2IDJN5vAxOTsdyG+ZsX0Bule4gZ33DpUaxcZDB3ATVNhm+Ok6Lt3JUVm+w8/WN4Z
0Ld2IbG25ls7YReh8lNBIrdFeMUJF6gWbZZoB8jctECe9ZjBMNx5JAXvzAlEJVdg
MF+jDC8FMMDwLP7mAfuIGwjoHJGbu5JsIzrb1RGsPadfInpmurKTdN88mp0/dH+8
EYNhqQhutgF2id4Fbj8cgX+4LKK1fzF62LQAUUyuVbpJ7rDTDbn3mdsK8MtYznKn
DZ3tJ4vPPkMAeuoDo9lHzuFdWA+lYzpHZM7d0/XUoZqrIs29mfu6algb1phYtj6c
iQIDAQAB
-----END PUBLIC KEY----- 

Generated hash : 
 8971128582297218747476502790184044331902692287615602258001491316291456083401


In [143]:
# hash2 = int.from_bytes(my_sha.digest(),byteorder='big')
# hashFromSignature = pow(signature, key.e, key.n)
# print("Signature valid:", hash2 == hashFromSignature)

Client re-hashes the public key and and compares with the hash that is sent. If they match the client proceeds to Step 5. Otherwise, the process ends here with an error message.

In [145]:
# Hash key 
my_sha = hashlib.sha256(pk_client)
hash_server = int.from_bytes(my_sha.digest(),byteorder='big')
# print("Sent to client: {public key, hash(public key)}. ",'\n'+'↓↓↓↓↓↓↓↓↓↓↓↓','\n')
# print("Public key :",'\n',pk_client.decode(),'\n')
# print("Generated hash :",'\n',hash_server)
if (hash_client==hash_server):
    print("hash match")
else:
    print("hash missmatch")

hash match
